### 其中将 balloon dataset 转化为 COCO 格式的代码如下所示。

In [2]:
import scripts.convert_coco

In [3]:
train_ann_file_path = '/openbayes/input/input0/balloon/train/via_region_data.json'
train_out_file_path = '/openbayes/input/output0/balloon/train_via_region_data.json'
train_image_prefix  = '/openbayes/input/input0/balloon/train/'

val_ann_file_path = '/openbayes/input/input0/balloon/val/via_region_data.json'
val_out_file_path = '/openbayes/input/output0/balloon/val_via_region_data.json'
val_image_prefix  = '/openbayes/input/input0/balloon/val/'

In [6]:
scripts.convert_coco.convert_balloon_to_coco(train_ann_file_path, train_out_file_path, train_image_prefix)
scripts.convert_coco.convert_balloon_to_coco(val_ann_file_path, val_out_file_path, val_image_prefix)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 61/61, 37.3 task/s, elapsed: 2s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 13/13, 35.9 task/s, elapsed: 0s, ETA:     0s


In [7]:
!ls /openbayes/input/output0/balloon/

train_via_region_data.json  val_via_region_data.json


### 准备配置文件

查询 mask r-cnn 的标准配置文件

In [9]:
!mim search mmdet --model "mask r-cnn"

--------------------------------------------------------------------------------
config id: mask_rcnn_convnext-t_p4_w7_fpn_fp16_ms-crop_3x_coco
architecture                                            convnext
coco/box_ap                                             46.2
coco/mask_ap                                            41.7
config                                                  configs/convnext/mask...
epochs                                                  36
model                                                   mask r-cnn
paper                                                   URL,Title
readme                                                  configs/mask_rcnn/REA...
training_data                                           coco
training_memory(GB)                                     7.3
training_resources                                      8x a100 gpus
training_techniques                                     adamw,mixed precision...
weight                                      

下载标准配置文件和 ckpt 文件

In [10]:
!mim download mmdet --config mask_rcnn_r50_fpn_fp16_1x_coco --dest ./config/_base_

processing mask_rcnn_r50_fpn_fp16_1x_coco...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 MiB 6.9 MB/s eta 0:00:0000:0100:01
Successfully downloaded mask_rcnn_r50_fpn_fp16_1x_coco_20200205-59faf7e4.pth to /output/mmdet-hw/config/_base_
Successfully dumped mask_rcnn_r50_fpn_fp16_1x_coco.py to /output/mmdet-hw/config/_base_


继承标准配置文件形成自己的配置

In [17]:
from mmcv import Config

config = Config.fromfile('config/mask_rcnn_r50_fpn_fp16_1x16b-50e_balloon.py')
print(config.pretty_text)  # 把已有的继承进来，再进行修改

model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)

### 准备训练模型

In [18]:
!mim train mmdet config/mask_rcnn_r50_fpn_fp16_1x16b-50e_balloon.py

/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Training command is /usr/local/bin/python /output/mmdet-hw/mmdetection/mmdet/.mim/tools/train.py config/mask_rcnn_r50_fpn_fp16_1x16b-50e_balloon.py --laun

### 测试训练的模型

In [13]:
import cv2
import mmcv
import numpy as np
import torch

from mmdet.apis import init_detector, inference_detector


def test_video(src_path, dst_path, cfg_path, pth_path):
    score_thr = 0.4
    video_reader = mmcv.VideoReader(src_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(
        dst_path, fourcc, video_reader.fps,
        (video_reader.width, video_reader.height))
    print(len(video_reader))

    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    model = init_detector(cfg_path, pth_path, device=device)
    for frame in video_reader:
        result = inference_detector(model, frame)
        mask = None
        masks = result[1][0]
        for i in range(len(masks)):
            if result[0][0][i][-1] >= score_thr:
                if not mask is None:
                    mask = mask | masks[i]
                else:
                    mask = masks[i]

        # 获取各通道mask像素

        masked_b = frame[:, :, 0] * mask
        masked_g = frame[:, :, 1] * mask
        masked_r = frame[:, :, 2] * mask
        masked = np.concatenate([masked_b[:,:,None],masked_g[:,:,None],masked_r[:,:,None]],axis=2)

        # frame转灰度图

        un_mask = 1 - mask
        frame_b = frame[:, :, 0] * un_mask
        frame_g = frame[:, :, 1] * un_mask
        frame_r = frame[:, :, 2] * un_mask
        frame = np.concatenate([frame_b[:, :, None], frame_g[:, :, None], frame_r[:, :, None]], axis=2).astype(np.uint8)
        frame = mmcv.bgr2gray(frame, keepdim=True)
        frame = np.concatenate([frame, frame, frame], axis=2)
        # mask加到灰度图中

        frame += masked

        # frame = model.show_result(frame, result, score_thr=score_thr)
        video_writer.write(frame)

    #   掩码计算->灰度图->灰度图*3->替换值

    video_writer.release()
    cv2.destroyAllWindows()

In [14]:
# import scripts.color_splash

src_path = "test/test_video.mp4"
dst_path = "result/result_video.mp4"  # mask and detect
config_path = "work_dirs/mask_rcnn/mask_rcnn_r50_fpn_fp16_1x16b-50e_balloon/mask_rcnn_r50_fpn_fp16_1x16b-50e_balloon.py"
pth_path = "work_dirs/mask_rcnn/mask_rcnn_r50_fpn_fp16_1x16b-50e_balloon/latest.pth"

# scripts.color_splash.test_video(src_path, dst_path, config_path, pth_path)
test_video(src_path, dst_path, config_path, pth_path)

150
load checkpoint from local path: work_dirs/mask_rcnn/mask_rcnn_r50_fpn_fp16_1x16b-50e_balloon/latest.pth


In [16]:
!pip install moviepy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 258.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110728 sha256=e43539b7daf79758b896a840bce2088a9a7f4263e7a396916bf9d02377df3336
  Stored in directory: /root/.cache/pip/wheels/e9/1e/f0/e1957cfe0900ee45b58986e568e5ff0b44a385cee00e4874f0
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


In [17]:
from moviepy.editor import *

clip = (VideoFileClip("result/result_video.mp4"))
clip.write_gif("result/result_video.gif")

MoviePy - Building file result/result_video.gif with imageio.
